## Video Demos

### Install av library

In [ ]:
!conda install av -c conda-forge -y

### Download and play video

In [ ]:
!curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/Intro.mp4 -o video.mp4

In [ ]:
%%HTML
<video width="320" height="240" controls>
  <source src="video.mp4" type="video/mp4">
</video>

### Count frames and show the 25th one

In [ ]:
import av
container = av.open('video.mp4')

%matplotlib inline
from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw

for frame in container.decode(video=0):
    if (frame.index == 25):
        img = frame.to_image()
        imshow(img)
frameCount = frame.index - 1
print(str(frameCount) + " frames")

## Face Tracking

### Download and play video

In [ ]:
!curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/vid.mp4 -o video.mp4

In [ ]:
%%HTML
<video width="320" height="240" controls>
  <source src="video.mp4" type="video/mp4">
</video>

### Start Async Search for Faces

In [ ]:
video_url="https://github.com/MicrosoftLearning/AI-Introduction/raw/master/files/vid.mp4"

videoURI = "REGION.api.cognitive.microsoft.com"
videoKey = "YOUR_KEY_HERE"

import http.client, urllib.request, urllib.parse, urllib.error, base64, json

headers = {
    # Request headers.
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': videoKey,
}

params = urllib.parse.urlencode({})

body = "{'url':'" + video_url + "'}"

try:
    # Execute the REST API call and get the response header.
    conn = http.client.HTTPSConnection(videoURI)
    conn.request("POST", "/video/v1.0/trackface?%s" % params, body, headers)
    response = conn.getresponse()
    oid = response.info()["Operation-Location"]
    conn.close()


except Exception as e:
    print('Error:')
    print(e)

# The operation is Async, so use this URL to check status
print(oid)

### Check Status

In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': videoKey,
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection(videoURI)
    conn.request("GET", oid + "?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    parsed = json.loads(data)
    print(parsed)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))



### Were faces detected?

In [ ]:
r = json.loads(parsed["processingResult"])
print(r['facesDetected']) 

### If so, when and where?

In [ ]:
ts = r['timescale']
fr = r['framerate']
width = r['width']
height = r['height']
for f in r['fragments']:
    if 'events' in f:
        s = f['start']
        secs = s/ts
        frameNo = secs * fr
        print(str(secs) + " seconds (frame:" + str(frameNo) + ")")
        e1 = f['events'][0]
        for face in e1:
            faceId =face['id']
            x = face['x'] * width
            y = face['y'] * height
            w = face['width'] * width
            h = face['height'] * height
            print("Face:" + str(faceId))
            print ("x:" + str(x) + ", y:" + str(y) + ", w:" + str(w) + ", h:" + str(h))
            break
        break


### Find the face

In [ ]:
%matplotlib inline
from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw

container = av.open('video.mp4')

for frame in container.decode(video=0):
    if (frame.index == frameNo):
        img = frame.to_image()
        break
draw = ImageDraw.Draw(img)
draw.line([(x, y),(x + w, y)],fill="red", width=5)
draw.line([(x + w ,y),(x + w, y + h)],fill="red" , width=5)
draw.line([(x + w, y + h),(x, y +h )],fill="red" , width=5)
draw.line([(x ,y + h),(x, y)],fill="red" , width=5)
imshow(img)